## CS310 Natural Language Processing
## Assignment 4. Long Short Term Memory (LSTM) Network for Named Entity Recognition (NER)

**Total points**: 50 + (10 bonus)

In this assignment, you will implement a Long Short Term Memory (LSTM) network for Named Entity Recognition (NER). 

Re-use the code in Lab 5.

### 0. Import Necessary Libraries

### 1. Build the Model

### 2. Train and Evaluate

In [ ]:
from utils import get_tag_indices_from_scores
from metrics import MetricsHandler

labels_str = ['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']
labels_int = list(range(len(labels_str)))
train_metrics = MetricsHandler(labels_int)

def train_model(model, data_loader, optimizer, loss_func, train_metrics, num_epochs=5, **kwargs):
    model.train()
    losses = []
    for epoch in range(num_epochs):
        running_loss = 0.0
        for batch in data_loader:
            optimizer.zero_grad()

            inputs, targets = batch
            output = model(inputs) # This is just a demo
            loss = loss_func(output, targets)

            loss.backward()
            optimizer.step()

            predictions = get_tag_indices_from_scores(output.detach().cpu().numpy())
            train_metrics.update(predictions, targets)
            running_loss += loss.item() * inputs.size(0)
        
        epoch_loss = running_loss / len(data_loader)
        losses.append(epoch_loss)

    return model, train_metrics, losses

def evaluate_model(model, data_loader, loss_func, eval_metrics, **kwargs):
    pass

### 3. Other Experiments